In [1]:
import sys
sys.path.append('/home/cryptobrahman/Own/football_competitions_research/own_modules/') 
sys.path.append('/mnt/KINGSTON_120/Own/football_competitions_research/own_modules/') 

In [28]:
import pickle
import pandas as pd
from pprint import pprint

from _astro_constants import AstrologicalConstants, AstrologicalPoints, TransformDoubleValues, ObjectsPrepare, AspectsPrepare, AspectsClearing

pd.set_option('display.max_columns', 200)  # or 1000
pd.set_option('display.max_rows', 200)  # or 1000
pd.set_option('display.max_colwidth', 100)  # or 199

In [4]:
%load_ext autoreload
%autoreload 2

In [73]:
# data = {'date':['2015/03/13', '2016/05/23', '2018/10/03'], 'utc_time':['14:00', '12:30', '00:15'], 'lon':['38n32', '08n07', '14s42'], 'lat':['8w54', '18e14','36w04']}
# df = pd.DataFrame(data)
# df

In [31]:
with open('pickle_files/df_with_cities_coords_26_05_2022', 'rb') as f:
    df_coords = pickle.load(f)

In [109]:
df_coords.shape

(39069, 45)

In [119]:
df_coords = df_coords[df_coords.country != 'USA'].copy()
df_coords.reset_index(drop=True, inplace=True)

In [137]:
print('shape - ', df_coords.shape[0])
df_coords.head()

shape -  38479


,game_id,ligue_header,comp_id,season_id,game_utc,game_title,goals,ligue,f_team,pos_f,pts_f,pos_s,pts_s,fs_pos_max,fs_pts_max,teams_ids,event_mins,event_desc,viewers,weath_temp,shots,saves,poss,corner,fouls,yl_crd,rd_crd,attacks,att_dang,tm_1,tm_x,tm_2,ls_25,mr_25,f_pow_bet,f_pow_pts,f_pow,desc,country,city,population,latitude,longitude,lat,lon
0,1687696,OBOS liga,668,301,26.05.2022 15:00,Skeid - Sandnes Ulf,0:1,thr,Skeid,16.0,3.0,4.0,18.0,"16 - 4, 16","3 - 18, 23",10294,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,Pre,Pre,fav_pre,Norway,Oslo,808690.0,59.916667,10.750000,59n55,10e45
1,1687694,OBOS liga,668,301,26.05.2022 16:00,Kongsvinger - Brann,1:3,thr,Kongsvinger,13.0,7.0,1.0,23.0,"13 - 1, 16","7 - 23, 23",None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,Pre,Pre,fav_pre,Norway,Kongsvinger,11147.0,60.200000,12.000000,60n12,12e0
2,1687692,OBOS liga,668,301,26.05.2022 16:00,Bryne - Stabaek,0:0,thr,Bryne,12.0,8.0,3.0,18.0,"12 - 3, 16","8 - 18, 23",None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,Pre,Pre,fav_pre,Norway,Bryne,NaN,58.733333,5.650000,58n43,5e39
3,1687698,OBOS liga,668,301,26.05.2022 16:00,Stjordals-Blink - Sogndal,1:3,thr,Stjordals-Blink,15.0,5.0,8.0,14.0,"15 - 8, 16","5 - 14, 23",8922,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,Pre,Pre,fav_pre,Norway,Stjørdalshalsen,9848.0,63.466667,10.933333,63n28,10e55
4,1692564,Division 1,713,301,26.05.2022 11:00,Ängelholms FF - Lindome,4:0,thr,Ängelholms FF,5.0,20.0,13.0,9.0,"5 - 13, 16","20 - 9, 24",7717,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,Fav,Fav,fav_pre,Sweden,Ängelholm,NaN,57.150000,15.150000,57n8,15e9


In [121]:
df             = df_coords[['game_utc', 'lon', 'lat']].copy()
df['date']     = df.game_utc.str[:10].str.split('.')
df['utc_time'] = df.game_utc.str[11:16]
df.date        = df.date.str[2] +'/'+ df.date.str[1] +'/'+ df.date.str[0]

In [122]:
df.drop(columns='game_utc', inplace=True)

In [123]:
df = df[['date', 'utc_time', 'lon', 'lat']]

In [138]:
df.head()

,date,utc_time,lon,lat
0,2022/05/26,15:00,10e45,59n55
1,2022/05/26,16:00,12e0,60n12
2,2022/05/26,16:00,5e39,58n43
3,2022/05/26,16:00,10e55,63n28
4,2022/05/26,11:00,15e9,57n8


### Get astrological objects

In [152]:
df_tmp = df.loc[11:11,].copy()

In [153]:
df_tmp

,date,utc_time,lon,lat
11,2022/05/25,13:00,93e1,56n39


In [149]:
df_tmp.lon = '93e1'

In [157]:
df_tmp.lat = '36n39'

In [159]:
# df['charts'] = AstrologicalPoints.charts_calculate(df_tmp, 'date', 'utc_time', 'lon', 'lat')

In [139]:
# df

### Calculate aspects

In [15]:
def aspects_calculate(df: pd.DataFrame, row_number: int):
    objects = ObjectsPrepare(df.iloc[row_number]).conbine_class_methods()
    tuple_list = df.transform_id[row_number]

    all_aspects = []

    houses_remove_objs = []
    moon_remove_objs   = ['Asc', 'Desc', 'MC', 'IC', 'Uranus', 'Neptune', 'Pluto', 'Chiron', 'North Node', 'South Node', 'Antes Moon','Antes Uranus', 
                          'Antes Neptune', 'Antes Pluto', 'Antes Chiron']
    main_remove_objs   = ['Asc', 'Desc', 'MC', 'IC', 'Moon', 'Antes Moon']
    houses_aspect_degs = {'Con': 0}
    main_aspect_degs   = {'Con': 0, 'Opp': 180}
    moon_aspect_degs   = AstrologicalConstants.aspekts_degrees
    house_ids          = AstrologicalConstants.houses

    house_objs, moon_objs, main_objs = objects.copy(),objects.copy(), objects.copy() 

    moon_asp_after  = AspectsPrepare.object_aspects('Moon', moon_objs, moon_remove_objs, moon_aspect_degs)
    moon_asp_before = AspectsPrepare.object_aspects('Moon', moon_objs, moon_remove_objs, moon_aspect_degs, before_point_asp='yes', before_orb=1)
    all_aspects.append(moon_asp_after)
    all_aspects.append(moon_asp_before)

    for h_obj in house_ids:
        house_asp_after  = AspectsPrepare.object_aspects(h_obj, house_objs, houses_remove_objs, houses_aspect_degs, before_point_asp='no', after_orb=3)
        house_asp_before = AspectsPrepare.object_aspects(h_obj, house_objs, houses_remove_objs, houses_aspect_degs, before_point_asp='yes', before_orb=3)
        all_aspects.append(house_asp_after)
        all_aspects.append(house_asp_before)

    for m_obj in objects:
        main_asp_after  = AspectsPrepare.object_aspects(m_obj.id, main_objs, main_remove_objs, main_aspect_degs, before_point_asp='no', after_orb=3)
        main_asp_before = AspectsPrepare.object_aspects(m_obj.id, main_objs, main_remove_objs, main_aspect_degs, before_point_asp='yes', before_orb=3)
        all_aspects.append(main_asp_after)
        all_aspects.append(main_asp_before)

    for asp in all_aspects:
        if asp != None and any(var['f_point'] == 'Asc' and var['s_point'] == 'Pluto' for var in asp):
            asc_pluto = asp
        else:  
            asc_pluto = []

    if asc_pluto == []:
        for obj in objects:
            if obj.id == 'Asc' or obj.id == 'Pluto':
                asc_pluto.append(obj) 

        asc_pluto_asp_after = AspectsPrepare.object_aspects('Asc', asc_pluto, houses_remove_objs, houses_aspect_degs, before_point_asp='no', after_orb=20)
        all_aspects.append(asc_pluto_asp_after)

    all_aspects_full = AspectsPrepare.transform_dict_list_type(all_aspects)

    AspectsClearing.remove_duplicate_aspects(all_aspects_full)
    AspectsClearing.remove_antes_unimportant_aspects(all_aspects_full)
    AspectsClearing.remove_node_opposition(all_aspects_full)
    AspectsClearing.remove_antes_with_own_nat_point_aspects(all_aspects_full)
    AspectsClearing.remove_mirror_antes_aspects(all_aspects_full, tuple_list)
    AspectsClearing.remove_mirror_houses_antes_conuction(all_aspects_full)

    for asp in all_aspects_full:
        asp['den_point'] = []

    AspectsPrepare.supplement_houses_and_parts_rulers_aspects(objects, all_aspects_full, tuple_list)
    AspectsPrepare.supplement_conv_stat_aspects(objects, all_aspects_full)
    AspectsPrepare.supplement_moon_complete_denide_weak_aspects(objects, all_aspects_full, tuple_list)
    AspectsPrepare.scale_orb_characteristic(all_aspects_full)
    AspectsPrepare.moon_scale_orb_characteristic(all_aspects_full)
    AspectsPrepare.aspects_rulers_characteristic(tuple_list, all_aspects_full)

    AspectsClearing.remove_houses_one_nodes_conuction(all_aspects_full)  

    return [all_aspects_full]

In [16]:
aspects = []
for x in range(df.shape[0]):
     aspects.append(aspects_calculate(df, x))  

df['aspects'] = aspects        

In [25]:
# pprint(aspects, sort_dicts=False)

In [27]:
# print(len(aspects))
# for x in aspects:
#     print(x)

In [19]:
# for x in objects:
#     if x.id == 'Mars':
#         x.lon = 1
#     if x.id == 'Pluto':
#         x.lon = 2      

In [20]:
#     Rules for aspekts:
#     Antes Moon - only for house conuction
#     Antes has only house rulers planets, higher planets and Parse Fortuna.

#     Moon aspects by specific points (houses rulers, there antes and fortuna parts with it antes):
#     'moon_conv_clear_compl' - any first aspect by parse fortuna and it antes, or first 'Con' aspect to houses rulers or any last aspect to houses rulers without 
#       any point aspects beefore
#     'moon_conv_compl' - any first aspect by parse fortuna and it antes, or if first 'Con' aspect to houses rulers or any last aspect to houses rulers with 
#       aspected beefore any point among active planets and only 'Con' type for all other points if their not in house rulers
#     'moon_conv_compl_weak' - any first aspect by parse fortuna and it antes or if first 'Con' aspect to houses rulers or any last houses rulers's aspect 
#       which was aspecting beefore parses or other houses rulers and last aspect antes's houses rulers which wasn't aspecting beefore parses or other houses rulers -------
#     'moon_conv' - all other points aspects
#     'moon_conv_denide' - if aspect sing: 'diff' or if aspect's 'tr_orb' > 5.1 
#     'moon_diver', 'moon_diver_weak' - for divergent Moon aspects.

#     Houses rulers and parts rulers aspects are aspecting own parts(houses):
#     'conv_compl', (conv_in_compl', conv_out_compl') - approuch parts(houses) rulers aspecting own parts(houses) without Moon's aspect at part 
#     'conv_denide', (conv_in_weak', conv_out_weak') - if parts(houses) rulers has other early conversation aspect(Moon include) or locate in different sings
#     'diver_weak', (diver_in_weak', diver_out_weak') - if aspect is divergent
#     'diver_denide' - if pars ruler has divergent aspect and it has previos con aspect

#     Other 'conv' and 'stat' aspects:
#     'conv_denide', 'stat_denide' - if points locate in different sings.
#     'conv_weak', 'stat_weak', ('in_conv_weak', 'out_conv_weak', 'in_stat_weak', 'out_stat_weak') - weakened aspects with the other long point 
#     located beetwen points longs. 

#     Remove antes aspects: 
#     If exist aspect between natural points - remove all unimportant aspects with anteses:
#       Stay - ('Mars' con 'Venus') or ('Mars' opp 'Venus')
#       Remove aspects ('Mars' opp 'Antes Venus', 'Mars' opp 'Antes Mars', ' Venus' opp 'Antes Mars', 'Venus' opp 'Antes Venus', 'Antes Mars' opp 'Antes Venus')
#     If exist mirror aspect between natural and antes point without aspect between natural points: 
#       ('Mars' con 'Antes Chiron') and ('Chiron' con 'Antes Mars') 
#       required stay the houses ruler's aspect, if aspect has two houses rulers or any other points - stay any.
#     Antes aspect by own planet without natural points aspect ('Mars' con 'Antes Mars')
#     Remove mirror houses antes aspects: ('Asc' con 'Mars') - ('Dsc' con 'Antes Mars').
#     Remove one mirror 'Moon Node' aspect by houses with 'weak' status from 'den_point' planet or any.